In [144]:
import pandas as pd
import numpy as np

academic_data = pd.read_csv('A:/Projects/Students/academic_data.csv')
behavioural_data = pd.read_csv("A:/Projects/Students/behavioral_data.csv")
extracurricular_data = pd.read_csv("A:/Projects/Students/extracurricular_data.csv")

In [78]:
#academic_data.head(5)
#behavioural_data.head(5)
#extracurricular_data.head(5)


In [157]:
#academic_data.isnull().sum()
#behavioural_data.isnull().sum()
#extracurricular_data.isnull().sum()

#merging the datasets and Aggregating the hours per week spent by the students in different activities
merged_data = pd.merge(academic_data, behavioural_data, on = 'Student_ID' )

extracurricular2_data= extracurricular_data.groupby('Student_ID').agg({'Hours_Per_Week':'sum'})

Merged_data = pd.merge(merged_data, extracurricular2_data, on='Student_ID')

#Merged_data.head(7)

#Merged_data.describe()


# Calculate the mean across specified columns for each student
Avg_Grade = Merged_data[['Grades', 'Attendance', 'Homework_Completion', 'Classroom_Attention', 'Interaction_With_Peers', 'Discipline_Incidents', 'Hours_Per_Week']].mean(axis=1)

# Add the calculated mean to the DataFrame
Merged_data['Avg_Grade'] = Avg_Grade

#Merged_data.head(10)
#Merged_data['Avg_Grade'].describe()

#Assigning Avg_Grade the Categorial Grades for visualization
def final_grade(Avg_Grade):
    if Avg_Grade >=  43.757769:
        return 'O (Outstanding)'
    elif ((Avg_Grade < 43.757769)and(Avg_Grade >= 38.786632)):
        return 'A (Excellent)'
    elif ((Avg_Grade < 38.786632)and(Avg_Grade >= 36.652989)):
        return 'B (Very Good)'
    elif ((Avg_Grade < 36.652989)and(Avg_Grade >= 34.747342)):
        return 'C (Good)'
    elif ((Avg_Grade < 34.747342)and(Avg_Grade >= 29.255739)):
        return 'D (Can Do Better)'
    else:
        return 'E (Poor Performance)'

#Creating a new column in named Final_Grades in Merged_data dataset using the function written above
Merged_data['Final_Grades']= Merged_data['Avg_Grade'].apply(final_grade)

Merged_data.head(5)

        


,Student_ID,Grades,Attendance,Homework_Completion,Classroom_Attention,Interaction_With_Peers,Discipline_Incidents,Hours_Per_Week,Avg_Grade,Final_Grades
0,1,79.967142,79.187065,60.953440,3.041588,3.728498,0,1.296422,32.596308,D (Can Do Better)
1,2,73.617357,77.614196,51.836068,6.462168,3.930923,1,6.141452,31.514595,D (Can Do Better)
2,3,81.476885,90.910756,55.401288,6.975545,7.226348,0,1.171350,34.737453,D (Can Do Better)
3,4,90.230299,92.661893,66.943032,10.000000,6.770939,0,7.596684,39.171835,A (Excellent)
4,5,72.658466,75.789654,90.129284,8.914468,5.431420,1,7.324185,37.321068,B (Very Good)


In [158]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#defining the features
features = Merged_data[['Grades','Attendance','Homework_Completion','Classroom_Attention','Interaction_With_Peers','Discipline_Incidents','Hours_Per_Week']]

#setting the target variable
target = Merged_data['Avg_Grade']

#splitting the data for training and testing
X_train, x_test, Y_train, y_test = train_test_split(features, target, test_size= 0.2, random_state = 42)

#feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
x_test = scaler.transform(x_test)

#training the model
Std_Prog_model = RandomForestRegressor(n_estimators=100, random_state=42)
Std_Prog_model.fit(X_train, Y_train)




RandomForestRegressor(random_state=42)

In [159]:
#Model Evaluation
from sklearn.metrics import mean_squared_error, r2_score

y_pred = Std_Prog_model.predict(x_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('mse:',mse)
print('r2:',r2)



mse: 0.35820278252978954
r2: 0.9527360260613223


In [112]:
# Feature importance
#importances = model.feature_importances_
#feature_names = features.columns
#importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances}).sort_values(by='Importance', ascending=False)

#importance_df

In [164]:
#Visualizing the progress of a particular student
import matplotlib.pyplot as plt

#Function to get the data of a particular student and visualize using the Student_ID given by user
def visualize_progress(student_id):
    student_data = Merged_data[Merged_data['Student_ID'] == student_id]
    if student_data.empty:
        print("Enter the valid Student_ID.")
        return

    # Extract components for the pie chart
    grades = student_data['Grades'].values[0]
    attendance = student_data['Attendance'].values[0]
    homework_completion = student_data['Homework_Completion'].values[0]
    classroom_attention = student_data['Classroom_Attention'].values[0]
    interaction_with_peers = student_data['Interaction_With_Peers'].values[0]
    discipline_incidents = student_data['Discipline_Incidents'].values[0]
    hours_per_week = student_data['Hours_Per_Week'].values[0]

    # Calculate the final grade
    final_grade = student_data['Final_Grades'].values[0]

    # Data for the pie chart
    labels = ['Grades', 'Attendance', 'Homework Completion', 'Classroom Attention', 
              'Interaction with Peers', 'Discipline Incidents', 'Hours per Week']
    sizes = [grades, attendance, homework_completion, classroom_attention, 
             interaction_with_peers, discipline_incidents, hours_per_week]
    colors = ['#ff9999','#66b3ff','#99ff99','#ffcc99','#c2c2f0','#ffb3e6','#c4e17f']

    # Create the pie chart
    fig, ax = plt.subplots(figsize=(10, 7))
    ax.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)
    ax.axis('equal')  

    plt.title(f'Student {student_id} Performance Distribution')
    plt.show()

    # Create a scorecard
    fig, ax = plt.subplots(figsize=(5, 3))
    ax.text(0.5, 0.5, f'Final Grade: {final_grade}', horizontalalignment='center', 
            verticalalignment='center', fontsize=24, fontweight='bold', color='blue')
    ax.axis('off')

    plt.title(f'Student {student_id} Scorecard')
    plt.show()

# Ask the user to enter a Student_ID
student_id = int(input("Enter the Student_ID: "))
visualize_progress(student_id)

Enter the Student_ID:  7777


Enter the valid Student_ID.
